In [7]:
import torch
from gensim.models import Word2Vec
import torch.nn as nn
import cv2

In [8]:
w2v = Word2Vec.load("bare_minimum")
w2v.wv["bear"].shape

(16,)

In [11]:
w2v.wv.__len__()

10

In [75]:
class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense1 = nn.Linear(16, 256)
        
        self.upsample1 = nn.ConvTranspose2d(1, 1, 3, 2, 1, 1)
        self.upsample2 = nn.ConvTranspose2d(1, 1, 3, 2, 1, 1)
        self.upsample3 = nn.ConvTranspose2d(1, 1, 3, 2, 1, 1)
        self.upsample4 = nn.ConvTranspose2d(1, 1, 3, 2, 1, 1)
        
        self.mesh = nn.Conv2d(4, 3, 3, padding=1)
        
    def forward(self, word_vec, simage):
        
        x = self.dense1(word_vec)
        x = nn.functional.relu(x)
        
        x = x.reshape((-1,1,16,16))
        
        x = self.upsample1(x)
        x = nn.functional.relu(x)
        x = self.upsample2(x)
        x = nn.functional.relu(x)
        x = self.upsample3(x)
        x = nn.functional.relu(x)
        x = self.upsample4(x)
        x = nn.functional.relu(x)
        
        x = torch.concat((simage, x), dim=1)
        print(x.shape)
        return self.mesh(x)
    
class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 3, 3, padding=1)
        self.conv2 = nn.Conv2d(3, 3, 3, padding=1)
        self.conv3 = nn.Conv2d(3, 3, 3, padding=1)
        
        self.pool1 = nn.MaxPool2d(2)
        self.conv4 = nn.Conv2d(3, 3, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2)
        self.conv5 = nn.Conv2d(3,6,3,padding=1)
        self.pool3 = nn.MaxPool2d(2)
        self.conv6 = nn.Conv2d(6,1,3,padding=1)
        self.pool4 = nn.MaxPool2d(2)
        
        self.flatten = nn.Flatten()
        self.classify1 = nn.Linear(272,64)
        self.classify2 = nn.Linear(64, 10)
        
        self.next = nn.Conv2d(3,3,5,padding=2)
        
    def forward(self, input, context):
        
        x = self.conv1(input)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = self.conv3(x)
        x = nn.functional.relu(x)
        
        c = self.pool1(x)
        c = self.conv4(c)
        c = nn.functional.relu(c)
        c = self.pool2(c)
        c = self.conv5(c)
        c = nn.functional.relu(c)
        c = self.pool3(c)
        c = self.conv6(c)
        c = nn.functional.relu(c)
        c = self.pool4(c)

        c = self.flatten(c)
        c = torch.concat((c, context), dim=1)
        c = self.classify1(c)
        c = nn.functional.relu(c)
        c = self.classify2(c)
        
        return nn.functional.relu(c)

In [76]:
enc = Encoder()
dec = Decoder()

In [77]:
a = enc(torch.from_numpy(w2v.wv['bear']), torch.zeros(1,3,256,256))
dec(a, torch.zeros(1,16))

torch.Size([1, 4, 256, 256])


tensor([[0.0962, 0.0850, 0.0000, 0.0000, 0.1239, 0.0498, 0.0000, 0.0000, 0.1029,
         0.0000]], grad_fn=<ReluBackward0>)

In [78]:
sentences = [["the", "bear", "is", "eating", "honey"],
             ["where", "is", "the", "hospital"],
             ["students", "are", "striking"]]

In [79]:
# define optim
crit_words = nn.CrossEntropyLoss()
crit_image = nn.MSELoss()

s = nn.ModuleList([enc,dec])

optim = torch.optim.Adam(s.parameters())

In [ ]:
## Training loop
epochs = 5
for epoch in range(epochs):
    for sent in sentences:
        optim.zero_grad()
        canvas = torch.zeros(1,3,256,256)
        for w in sent:
            canvas = enc(torch.from_numpy(w2v.wv[w]), canvas)
        # replace zeros with indended image for sentence
        loss = crit_image(torch.zeros_like(canvas), canvas)
        context = torch.zeros(1,16)
        outs = []
        for w in sent:
            word = dec(canvas, context)
            context = word
            loss = loss + crit_words(word, torch.tensor([w2v.wv.key_to_index[w]]))
            outs.append(word)
        loss.backward()
        optim.step()

torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1, 4, 256, 256])
torch.Size([1,